# Auto Portrait Collector

Collecting portraits of celebrities manually is tedious. This notebook is an attempt to delegate this task to a [browser-use](https://github.com/browser-use/browser-use)ing agent.

## Dependencies

In [1]:
!wget -qO- https://astral.sh/uv/install.sh | sh

downloading uv 0.6.17 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


In [2]:
!uv venv

Using CPython 3.11.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


In [3]:
!source .venv/bin/activate

In [4]:
!uv pip install browser-use

Using Python 3.11.12 environment at: /usr
Audited 1 package in 224ms


In [5]:
!uv run playwright install --with-deps

Installing dependencies...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 261 kB in 7s (40.1 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list

## Setup

In [6]:
from typing import List
from langchain_openai import ChatOpenAI
from browser_use import Agent, Browser, BrowserConfig, Controller
from browser_use.browser.context import BrowserContextConfig, BrowserContext
from pydantic import BaseModel

from google.colab import userdata
from dotenv import load_dotenv
import asyncio
import os

In [7]:
openai_api_key = userdata.get('OPENAI_API_KEY')
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
load_dotenv()

False

In [8]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [9]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BR0OPHCyhfKr97oZ4BLtEzi9l6ucR', 'finish_reason': 'stop', 'logprobs': None}, id='run-24d7075b-aaf1-45e8-8152-53fc2becc244-0', usage_metadata={'input_tokens': 31, 'output_tokens': 6, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
browser = Browser(
    config = BrowserConfig(
        headless=True
    )
)
config = BrowserContextConfig(
    allowed_domains=['pinterest.com'],
)
context = BrowserContext(browser=browser, config=config)

In [11]:
class Image(BaseModel):
	description: str
	url: str

class Images(BaseModel):
	images: List[Image]

controller = Controller(output_model=Images)

In [12]:
async def get_image_urls(target: str) -> Images:
    task = f'Go to https://www.pinterest.com/ideas/ and find at least three good portrait images of {target}. ' \
        'Prefer smiling images. For good images get their html src attribute. ' \
        'Dont click on images or follow their links, just get the src attribute of the thumbnail.'

    agent = Agent(
        task=task,
        llm=llm,
        controller=controller,
        browser_context=context,
    )
    history = await agent.run()

    result = history.final_result()

    if result:
        parsed: Images = Images.model_validate_json(result)
        return parsed
    else:
        return None


In [13]:
results = []

In [14]:
async def collect_images(target: str):
  images = await get_image_urls(target)
  results.append((target, images))

  for image in images.images:
    print(f'{image.url}\t{image.description}')

## Collect

In [15]:
await collect_images('Angourie Rice')

/usr/local/lib/python3.11/dist-packages/browser_use/agent/service.py:321: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace

https://i.pinimg.com/236x/a3/42/f8/a342f8bdc069686e217d59e9eafd5051.jpg	Portrait of Angourie Rice
https://i.pinimg.com/236x/d8/f8/9c/d8f89ca31b9462ed00ff83df437ec7cc.jpg	Portrait of Angourie Rice
https://i.pinimg.com/236x/a1/c8/65/a1c865bd91443a54416a975c820d6b67.jpg	Portrait of Angourie Rice


In [16]:
await collect_images('Emma Watson')

https://i.pinimg.com/236x/a8/5a/ea/a85aea6da0c6d02d38db0c9d0193d76d.jpg	Portrait of Emma Watson
https://i.pinimg.com/236x/8b/31/e4/8b31e484f8d4f48f09d449cb441d4fc8.jpg	Portrait of Emma Watson
https://i.pinimg.com/236x/17/57/88/1757884fc682152156babed90a47f2ee.jpg	Portrait of Emma Watson


In [17]:
await collect_images('Jenna Ortega')

https://i.pinimg.com/236x/53/e1/03/53e10331fc2ace135c8fe385a0fae841.jpg	Jenna Ortega portrait image 1
https://i.pinimg.com/236x/f9/bb/b4/f9bbb49cc8481bfc20a2931da09d79ca.jpg	Jenna Ortega portrait image 2
https://i.pinimg.com/236x/fd/cc/5a/fdcc5a9a43da6800fc9e7554e693bc10.jpg	Jenna Ortega portrait image 3


## Output

In [18]:
from IPython.display import HTML
html_output = ""
for target, images in results:
  html_output += f"""
  <div>
    <p>{target}</p>
    <div style="display: flex; gap: 10px;">
  """
  for image in images.images:
    html_output += f'<img src="{image.url}" height="200">'
  html_output += """
    </div>
  </div>
  """

if not html_output:
  html_output = "<p>No images found.</p>"

display(HTML(html_output))